<a href="https://colab.research.google.com/github/m-adeleke1/Association_of_Data_Scientists/blob/main/Agent_Memory_Using_MEM0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Library Installation

In [6]:
# !pip install pyautogen autogen-agentchat mem0ai
# !pip install -q --upgrade "pyautogen"
# !pip install -q --upgrade "mem0ai"

# import sys
# print("Using Python:", sys.executable)

# Force-install into THIS interpreter:
# !{sys.executable} -m pip install -q --upgrade pyautogen mem0ai

# import sys
# !{sys.executable} -m pip uninstall -y autogen autogen-agentchat autogen-core || true
# !{sys.executable} -m pip install -q --upgrade pyautogen

# import sys

# 1) Clean out anything modular/new
# !{sys.executable} -m pip uninstall -y autogen autogen-agentchat autogen-core || true

# 2) Install the legacy API (0.2.x line)
# !{sys.executable} -m pip install -U "pyautogen~=0.2.0"

import sys
# Remove modular/new packages if present
!{sys.executable} -m pip uninstall -y autogen autogen-core autogen-agentchat || true

# Make sure NumPy is consistent (force reinstall)
!{sys.executable} -m pip install -q --upgrade --force-reinstall --no-cache-dir "numpy>=2.0"

# Install dependencies required by legacy AutoGen 0.2.x
!{sys.executable} -m pip install -q --upgrade "ray>=2.30" "flaml>=2.1.0"

# Finally install legacy AutoGen (0.2.x line)
!{sys.executable} -m pip install -q --upgrade "pyautogen~=0.2.0" "mem0ai"

import importlib.util, sys

print("autogen spec:", importlib.util.find_spec("autogen"))
print("mem0 spec:", importlib.util.find_spec("mem0"))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 59.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pyautogen 0.2.35 requires numpy<2,>=1.17.0, but you have numpy 2.3.2 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.2 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.2 which is incompatib

### API Initialisation

In [1]:
from google.colab import userdata
import os

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['MEM0_API_KEY'] = userdata.get('MEM0_API_KEY')

### Agent Configuration with MemO

In [2]:
from autogen import ConversableAgent
from mem0 import MemoryClient

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [3]:
agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    code_execution_config=False,
    function_map=None,
    human_input_mode="NEVER",
)

memory = MemoryClient()

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:151: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


### Providing context for memory saving

In [4]:
conversation = [
  {
    "role": "assistant",
    "content": "Hello, I’m MedCare's virtual assistant.\n\nThank you for trusting MedCare Health System. How can I assist you today?"
  },
  {
    "role": "user",
    "content": "I've been having recurring chest pain and would like to get admitted for observation."
  },
  {
    "role": "assistant",
    "content": "I’m really sorry to hear that you’re experiencing this. Your health is our priority, and I’m here to help you every step of the way.\n\nTo get started, may I have your full name and date of birth, so I can access your patient profile?"
  },
  {
    "role": "user",
    "content": "Name: John Doe, DOB: 03/14/1985"
  },
  {
    "role": "assistant",
    "content": "Thank you, John. I’ve located your record in our system.\n\nBefore proceeding, may I also have the email address or phone number associated with your MedCare account for verification?"
  },
  {
    "role": "user",
    "content": "john.doe@email.com"
  },
  {
    "role": "assistant",
    "content": "Perfect, thank you for confirming your details.\n\nJohn, we take symptoms like chest pain very seriously. I can help you schedule a direct admission to our observation unit.\n\nMay I ask when you first began experiencing these symptoms and whether you have any history of heart-related conditions?"
  },
  {
    "role": "user",
    "content": "It started about three days ago. No previous heart conditions that I know of."
  },
  {
    "role": "assistant",
    "content": "Thanks for the information. I’ll include this in your intake form for the clinical team.\n\nSince this is a recurring issue and not an emergency at the moment, I recommend we schedule a priority admission tomorrow for observation and testing.\n\nWould you like me to go ahead and confirm the admission for tomorrow?"
  },
  {
    "role": "user",
    "content": "Yes, please."
  },
  {
    "role": "assistant",
    "content": "Thank you. I’m confirming an admission for tomorrow.\n\nOur patient coordinator will contact you within the next 2 hours with further instructions and to assist with pre-admission paperwork.\n\nIs there anything else I can help you with right now, John?"
  },
  {
    "role": "user",
    "content": "No, that's all. Thanks a lot!"
  },
  {
    "role": "assistant",
    "content": "You’re very welcome. Wishing you a quick recovery and we're here for you whenever you need us.\n\nTake care, John."
  }
]

memory.add(messages=conversation, user_id="patient_admission_bot")

/usr/local/lib/python3.12/dist-packages/mem0/client/utils.py:21: DeprecationWarning: output_format='v1.0' is deprecated therefore setting it to 'v1.1' by default. Check out the docs for more information: https://docs.mem0.ai/platform/quickstart#4-1-create-memories
  return func(*args, **kwargs)


{'results': [{'id': '6e9b29f0-c47e-4492-b461-17f1fccba063',
   'event': 'ADD',
   'memory': 'User Name is John Doe',
   'structured_attributes': {'day': 25,
    'hour': 3,
    'year': 2025,
    'month': 8,
    'minute': 12,
    'quarter': 3,
    'is_weekend': False,
    'day_of_week': 'monday',
    'day_of_year': 237,
    'week_of_year': 35}},
  {'id': '0b659b9e-f2c2-414d-80da-3b8fd0eda37c',
   'event': 'ADD',
   'memory': 'User DOB is 03/14/1985',
   'structured_attributes': {'day': 25,
    'hour': 3,
    'year': 2025,
    'month': 8,
    'minute': 12,
    'quarter': 3,
    'is_weekend': False,
    'day_of_week': 'monday',
    'day_of_year': 237,
    'week_of_year': 35}},
  {'id': '9231931a-5d49-4c10-a568-04fa723f2ef7',
   'event': 'ADD',
   'memory': 'User Email is john.doe@email.com',
   'structured_attributes': {'day': 25,
    'hour': 3,
    'year': 2025,
    'month': 8,
    'minute': 12,
    'quarter': 3,
    'is_weekend': False,
    'day_of_week': 'monday',
    'day_of_year': 237

### Checking the agent

In [5]:
data = "When is my admission for observation scheduled?"

relevant_memories = memory.search(data, user_id="patient_admission_bot")
flatten_relevant_memories = "\n".join([m["memory"] for m in relevant_memories])

prompt = f"""Answer the user question considering the memories. Keep answers clear and concise.
Memories:
{flatten_relevant_memories}
\n\n
Question: {data}
"""

reply = agent.generate_reply(messages=[{"content": prompt, "role": "user"}])
print(reply)

/usr/local/lib/python3.12/dist-packages/autogen/logger/logger_utils.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S.%f")


Your admission for observation is scheduled for tomorrow.


### Multi-agent Conversation with memory retrieval

In [6]:
manager = ConversableAgent(
    "manager",
    system_message="You are a medical support who helps in resolving patient issues.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",
)

patient_bot = ConversableAgent(
    "patient_bot",
    system_message="You are a medical support bot who gathers information on patient problems. Keep answers clear and concise.",
    llm_config={"config_list": [{"model": "gpt-4o", "api_key": os.environ.get("OPENAI_API_KEY")}]},
    human_input_mode="NEVER",
)

In [7]:
data = "What is the date of birth of patient named John Doe?"

relevant_memories = memory.search(data, user_id="patient_admission_bot")
flatten_relevant_memories = "\n".join([m["memory"] for m in relevant_memories])

prompt = f"""
Context:
{flatten_relevant_memories}
\n\n
Question: {data}
"""

result = manager.send(prompt, patient_bot, request_reply=True)

manager (to patient_bot):


Context:
User Name is John Doe
User Email is john.doe@email.com
User DOB is 03/14/1985
User Started experiencing symptoms about three days ago
User Is scheduled for admission tomorrow for observation and testing
User Has no previous heart conditions
User Has been having recurring chest pain



Question: What is the date of birth of patient named John Doe?


--------------------------------------------------------------------------------
patient_bot (to manager):

John Doe's date of birth is 03/14/1985.

--------------------------------------------------------------------------------
